# Deep Learning for Violence Detection in Surveillance Videos

## Development environment setup

In [ ]:
#@title Colab development environment settings
%%shell
if [ -n "${COLAB_RELEASE_TAG}" ]; then
    git clone -b development https://github.com/MigeoDaSelva/DeepLVioDeSurVideos.git \
    && echo -e "\n\nOkay!\n\n" \
    || echo -e "\n\nFAILED!\n\n";
    python3 -m pip install --upgrade pip;
    python3 -m pip install -r /content/DeepLVioDeSurVideos/requirements.txt;
    kill -9 pid "${PPID}";
fi

In [1]:
#@title Import libraries
from pprint import pprint
import sys
import os

In [2]:
#@title System path manager
%reload_ext autoreload
%autoreload 2

project_name = "DeepLVioDeSurVideos"

working_directory = os.popen("echo $PWD").read().rstrip()

project_root_path = f"{working_directory}/{project_name}" \
    if project_name not in working_directory else working_directory

if str(project_root_path) not in sys.path:
    sys.path.insert(0, str(project_root_path))

pprint(f"Environment paths: {sys.path}")

("Environment paths: ['/workspaces/DeepLVioDeSurVideos', "
 "'/usr/lib/python311.zip', '/usr/lib/python3.11', "
 "'/usr/lib/python3.11/lib-dynload', '', "
 "'/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages']")


In [3]:
#@title Import project stuff
from src.application.evaluation_application import EvaluationApplication
from src.application.execution_application import ExecutionApplication
from src.application.startup_application import StartupApplication
from configs import settings

In [ ]:
#@title Colab resource settings
if settings.COLAB_ENV: 
    from google.colab import output
    from google.colab import drive
    output.enable_custom_widget_manager()
    drive.mount("/content/drive", force_remount=True)

In [4]:
#@title Define simulation scope
simulation_scope_settings = StartupApplication.get_scope_manager()

In [6]:
#@title Select data settings
data_settings = StartupApplication.get_data_manager()

IntSlider(value=4, description='Batch size: ', layout=Layout(height='40px', width='auto'), max=1000, min=1, st…

IntSlider(value=30, description='Number of frames: ', layout=Layout(height='40px', width='auto'), min=1, style…

IntSlider(value=224, description='Image height: ', layout=Layout(height='40px', width='auto'), max=2000, min=1…

IntSlider(value=224, description='Image width: ', layout=Layout(height='40px', width='auto'), max=2000, min=1,…

IntSlider(value=3, description='Number of channels: ', layout=Layout(height='40px', width='auto'), max=10, min…

In [7]:
#@title Select approach settings
approach_settings = StartupApplication.get_approach_manager()

Dropdown(description='Approach options: ', layout=Layout(height='40px', width='auto'), options=(('(2+1)D-CNN',…

Dropdown(description='Optimizer options: ', index=2, layout=Layout(height='40px', width='auto'), options=('SGD…

Dropdown(description='Activation options: ', layout=Layout(height='40px', width='auto'), options=('relu', 'sig…

Dropdown(description='Padding options: ', layout=Layout(height='40px', width='auto'), options=('same', 'valid'…

IntSlider(value=30, description='Epochs: ', layout=Layout(height='40px', width='auto'), max=300, min=1, style=…

FloatLogSlider(value=32.0, base=2.0, description='Filters: ', layout=Layout(height='40px', width='auto'), max=…

FloatSlider(value=0.2, continuous_update=False, description='Dropout: ', layout=Layout(height='40px', width='a…

FloatLogSlider(value=0.01, description='Learning rate: ', layout=Layout(height='40px', width='auto'), max=-1.0…

Accordion(children=(Tab(children=(VBox(children=(Checkbox(value=False, description='Disabled', indent=False),)…

In [8]:
#@title Show selected values

print(
    "\nSimulation scope sttings:\n",
    f"Iterations: {simulation_scope_settings.iterations}",
    f"folds: {simulation_scope_settings.folds}",
    sep="\n"
)

print(
    "\nSelected data attributes:\n",
    f"Number of channels: {data_settings.n_channels}",
    f"Number of frames: {data_settings.n_frames}",
    f"Image height: {data_settings.image_height}",
    f"Image width: {data_settings.image_width}",
    f"Batch size: {data_settings.batch_size}",
    sep = "\n"
)

print(
    "\nSelected model attributes:\n",
    f"Callbacks: {[callback.__class__.__name__ for callback in approach_settings.callbacks]}",
    f"Number of output initial filters: {approach_settings.filters}",
    f"Pooling Strides: {approach_settings.pooling_strides}",
    f"Activation function: {approach_settings.activation}",
    f"Padding approach: {approach_settings.padding}",
    f"Kernel size: {approach_settings.kernel_size}",
    f"Number of epochs: {approach_settings.epochs}",
    f"Learning rate: {approach_settings.learning_rate}",
    f"Optimizer: {approach_settings.optimizer}",
    f"Dropout: {approach_settings.dropout}",
    sep = "\n"
)


Simulation scope sttings:

Iterations: [1]
folds: [1]

Selected data attributes:

Number of channels: 3
Number of frames: 30
Image height: 224
Image width: 224
Batch size: 4

Selected model attributes:

Callbacks: ['ModelCheckpoint', 'CSVLogger', 'EarlyStopping', 'LearningRateScheduler']
Number of output initial filters: 64
Pooling Strides: (2, 2, 2)
Activation function: relu
Padding approach: same
Kernel size: (3, 3, 3)
Number of epochs: 30
Learning rate: 0.01
Optimizer: Adam
Dropout: 0.2


In [9]:
#@title build pipeline
pipeline = ExecutionApplication.get_pipeline(
    simulation_scope_settings=simulation_scope_settings,
    data_settings=data_settings,
    approach_settings=approach_settings
)

In [10]:
#@title Run pipeline
pipeline.run()

Epoch 1/30
160/160 [==============================] - 5067s 30s/step - loss: 161.0520 - accuracy: 0.0953 - val_loss: 6.1539 - val_accuracy: 0.0562 - lr: 0.0010
Epoch 2/30
160/160 [==============================] - 4785s 30s/step - loss: 3.5757 - accuracy: 0.1047 - val_loss: 2.6338 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 3/30
160/160 [==============================] - 4552s 28s/step - loss: 2.6457 - accuracy: 0.1094 - val_loss: 2.6250 - val_accuracy: 0.1500 - lr: 0.0010
Epoch 4/30
160/160 [==============================] - 4540s 28s/step - loss: 2.6225 - accuracy: 0.1266 - val_loss: 2.6091 - val_accuracy: 0.1500 - lr: 0.0010
Epoch 5/30
138/160 [========================>.....] - ETA: 8:41 - loss: 2.6214 - accuracy: 0.1504

In [10]:
#@title Evaluate
evaluation_application = EvaluationApplication(
    approach=pipeline.approach, 
    data_settings=data_settings, 
    simulation_scope_settings=simulation_scope_settings,
)
evaluation_application.evaluate()

50/50 [==============================] - 7s 133ms/step


In [11]:
evaluation_application.calculate_classification_metrics()

/workspaces/DeepLVioDeSurVideos/src/engine/metrics_calculator.py:29: RuntimeWarning: invalid value encountered in scalar divide
  precision_value = tp[i] / (tp[i] + fp)
/workspaces/DeepLVioDeSurVideos/src/engine/metrics_calculator.py:38: RuntimeWarning: invalid value encountered in scalar divide
  (precision_value * recall_value) / (precision_value + recall_value)


MetricsResult(metrics=   Unnamed: 0     Metric  Abuse  Arrest  Arson  Assault  Burglary  Explosion  \
0           0  precision    NaN     NaN    NaN      NaN       NaN        NaN   
1           1     recall    0.0     0.0    0.0      0.0       0.0        0.0   
2           2   f1_score    NaN     NaN    NaN      NaN       NaN        NaN   

   Fighting  Normal  RoadAccidents  Robbery  Shooting  Shoplifting  Stealing  \
0       NaN     NaN       0.151515      0.0       NaN          NaN       NaN   
1       0.0     0.0       1.000000      0.0       0.0          0.0       0.0   
2       NaN     NaN       0.263158      NaN       NaN          NaN       NaN   

   Vandalism  
0        NaN  
1        0.0  
2        NaN  , output=Output())

In [13]:
evaluation_application.build_confusion_matrix()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x084\x00\x00\x084\x08\x06\x00\x00\x00LT\xb4\xc9\x00\…

GraphicalResult(image_as_bytes=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x084\x00\x00\x084\x08\x06\x00\x00\x00LT\xb4\xc9\x00\x00\x009tEXtSoftware\x00Matplotlib version3.8.0, https://matplotlib.org/5\xb1\xb5\xab\x00\x00\x00\tpHYs\x00\x00\t:\x00\x00\t:\x01\xf0d\x92J\x00\x01\x00\x00IDATx\x9c\xec\xddy\\\x95e\xfe\xff\xf17\xab\x08(\n\nn\xb8\xafi\x9ak{\x9a\x9a\x96Q.\xed\xa6e\xd3\xcc7\x1b\xb3l\x9bJg*\xa7Ek\xa6U+\xb3eLs\x9a\x16\xa5\x10M\xd3R\xd1\xa9T\xdc-\xb7\\\x12w\x05\x11\x01\x81\x83\x87\xdf\x1f\xfdd\xba9\xa0\xe7\x1c\x0e\\\xf7\x81\xd7\xf3\xf1\xe01\x8f\xfb\xc3}]\xf7\xe5\xdb\xfb\xdbW\xe0\xc3u\x05\x14\x15\x15\x15\t\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0F\x02M/\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0$\x1a\x1a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\xed\xd0\xd0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00l\x87\x86\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`;44\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xdb\xa1\xa1\x01\x00\x00\x00\x00\x00\x00\x00\x00\x0

<Figure size 3500x3500 with 0 Axes>